<h1>Capstone Project - Week 3</h1>
<em>Alex Lynn</em>

### Imports

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import urllib.request
from bs4 import BeautifulSoup

### Get Data From Website

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = urllib.request.urlopen(url).read()

#### Using BeautifulSoup
I will now load the website from the above URL

In [3]:
soup = BeautifulSoup(data, 'lxml')

In [4]:
table = soup.find('table', class_='wikitable sortable')

#### Adding Data to Arrays

In [5]:
postalcode = []
borough = []
neighbourhood = []

for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        postalcode.append(cells[0].find(text=True))
        borough.append(cells[1].find(text=True))
        neighbourhood.append(cells[2].find(text=True))


#### Add Arrays to Pandas DataFrame

In [6]:
df = pd.DataFrame(postalcode,columns=['PostalCode'])
df['Borough'] = borough
df['Neighbourhood'] = neighbourhood

df.shape

(180, 3)

### Data Processing

In the following cell I am removing whitespace and \\n from the data we have retrieved.<br>
Then finally I have dropped any Boroughs' which were Not Assigned.

In [7]:
# Trim whitespace
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Remove \n from columns
df.replace(r'\n',' ', regex=True, inplace=True) 

# Change all Not assigned in Neighbourhood to Borough
df['Neighbourhood'].replace('Not Assigned', df['Borough'], inplace=True)

# Remove Not Assigned from Borough
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 3)

#### 77 Rows have been removed as they did not have a borough assigned.

## Add Longitude and Latitude to DataFrame

#### Install GeoCoder

In [9]:
!pip3 install geocoder

Defaulting to user installation because normal site-packages is not writeable


#### Imports

In [10]:
import geocoder

#### Loop through all post_codes

In [11]:
# initialize your variable to None
# lat_lng_coords = None

# loop until you get the coordinates
# while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(postalcode))
  #lat_lng_coords = g.latlng

g = geocoder.google('Parkwoods, Toronto, Ontario')
g.latlng
lat_lng_coords = g.latlng
print(lat_lng_coords)
    

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

None


### Geocoder would not pass any variable back, imported the CSV instead

In [12]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
print(df_geo.shape)
df_geo.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Merge the two DataFrames based on Postal Codes

In [13]:
# Merge df and df_geo
merged = pd.merge(left=df, right=df_geo, left_on='PostalCode', right_on='Postal Code')
merged.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A,43.662301,-79.389494


#### Drop Postal Code from Merge

In [14]:
try:
    merged.drop('Postal Code', axis=1, inplace = True)
except:
    print('Postal Code already dropped')

In [16]:
merged.head(11)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
